# Array Broadcasting
## A Simple Example of Array Broadcasting
In this section, we will discuss the NumPy's array broadcasting mechanism for performing vectorized mathematical operations on arrays of *unequal* shape. Let's immediately take an example into consideration to see array broadcasting in action. Suppose you have a grade book for 6 students,each of whom have taken 3 exams; naturally, you store these scores in a shape-(6,3) array:

```python
# grades for 6 students who have taken 3 exams
>>> import numpy as np
>>> grades
array([[ 0.79,  0.84,  0.84],
       [ 0.87,  0.93,  0.78],
       [ 0.77,  1.  ,  0.87],
       [ 0.66,  0.75,  0.82],
       [ 0.84,  0.89,  0.76],
       [ 0.83,  0.71,  0.85]])
```

We might be interested to see how each of these scores compare to the mean score for that specific exam. Based on our discussion from the last section, we can easily compute the mean-score for each exam (rounded to 2 decimal places):

```python
# compute the mean score for each exam (rounded to 2 decimal places)
>>> mean_exam_scores = grades.mean(axis=0)
>>> mean_exam_scores = np.round(mean_exam_scores, 2)
>>> mean_exam_scores
array([ 0.79,  0.85,  0.82])
```

`grades` is a shape-(6,3) array and `mean_exam_scores` is a shape-(3,) array, and we want to compute the offset of each exam score from its respective mean. At first glance, it seems like we will have to loop over each row of our `grades` array and subtract from it the `mean_exam_scores`, to compute the offset of each exam score from the respective mean-score:
```python
# Using a for-loop to compute score offsets.

# Shape-(6,3) array that will store (score - mean) for each
# exam score.
score_offset = np.zeros_like(grades)

# iterates over each row of `grades`
for n, scores_per_student in enumerate(grades):
    # `scores_per_student` is a shape-(3,) array of exam scores
    # for a given student. This matches the shape of
    # `mean_exam_scores`, thus we can perform this subtraction
    score_offset[n] = scores_per_student - mean_exam_scores
```

Given our discussion of vectorized operations from the last section, you should recoil at the sight of a for-loop in code that is performing array-arithmetic. We might as well get out our abacuses and spreadsheets at this point. Fortunately, NumPy provides a mechanism called array broadcasting, which allows us to compute these offsets in a concise, vectorized way:
```python
# Using broadcasting to subtract a shape-(3,) array
# from a shape-(6,3) array.
>>> score_offset = grades - mean_exam_scores
>>> score_offset
array([[ 0.  , -0.01,  0.02],
       [ 0.08,  0.08, -0.04],
       [-0.02,  0.15,  0.05],
       [-0.13, -0.1 ,  0.  ],
       [ 0.05,  0.04, -0.06],
       [ 0.04, -0.14,  0.03]])
```


When you invoke `grades - mean_exam_scores`, NumPy will recognize that `mean_exam_scores` has the same shape as each row of `grades` and thus it will apply the subtraction operation on *each* row of `grades` with `mean_exam_scores`. What has transpired here is an example of *array broadcasting*. NumPy has effectively replicated the content of `mean_exam_scores` such that it is now a shape-(6,3) array, which is thus compatible for subtraction with `grades`. That is, the content of `mean_exam_scores` has effectively been *broadcasted* to fill a shape-(6,3) array:


\begin{equation}
\left( \begin{array}{*{3}{X}}
  0.79 & 0.84 & 0.84 \\
  0.87 & 0.93 & 0.78 \\
  0.77 & 1.00 & 0.87 \\
  0.66 & 0.75 & 0.82 \\
  0.84 & 0.89 & 0.76 \\
  0.83 & 0.71 & 0.85
\end{array} \right)
% 
- \left( \begin{array}{*{3}{X}}
  0.79 & 0.85 & 0.82
\end{array}\right)
% 
\rightarrow \left( \begin{array}{*{3}{X}}
  0.79 & 0.84 & 0.84 \\
  0.87 & 0.93 & 0.78 \\
  0.77 & 1.00 & 0.87 \\
  0.66 & 0.75 & 0.82 \\
  0.84 & 0.89 & 0.76 \\
  0.83 & 0.71 & 0.85
\end{array} \right)
%
- \left( \begin{array}{*{3}{X}}
  0.79 & 0.85 & 0.82 \\
  0.79 & 0.85 & 0.82 \\
  0.79 & 0.85 & 0.82 \\
  0.79 & 0.85 & 0.82 \\
  0.79 & 0.85 & 0.82 \\
  0.79 & 0.85 & 0.82
\end{array}\right)
\
\end{equation}


The word "effectively" is used here because NumPy doesn't really create this broadcasted version of `mean_exam_scores` behind the scenes; it is able to do the necessary computations without having to redundantly copy the contents of `mean_exam_scores` into a shape-(6,3) array. Doing so would be a waste of memory and computation. That being said, this replication process conveys exactly the mathematics of broadcast operations between arrays; thus the preceding diagram reflects how you should always envision broadcasting.  

<div class="alert alert-info"> 

**Definition: Array Broadcasting**: 

**Array broadcasting** is a mechanism used by NumPy to permit vectorized mathematical operations between arrays of unequal, but compatible shapes. Specifically, an array will be treated as if its contents have been replicated along appropriate the dimensions, such that the shape of this new, higher-dimensional array suites the mathematical operation being performed.     

</div>


NumPy provides the function [broadcast_to](https://docs.scipy.org/doc/numpy/reference/generated/numpy.broadcast_to.html#numpy.broadcast_to), which can be used to broadcast an array to a specified shape. This can help us build our intuition for broadcasting. Let's broadcast a shape-(3,4) array to a shape-(2,3,4) array:

```python
# Using `np.broadcast_to`; here we broadcast a 
# shape-(3,4) array to a shape-(2,3,4) array.
>>> x = np.array([[ 0,  1,  2,  3],
                  [ 4,  5,  6,  7],
                  [ 8,  9, 10, 11]])

>>> np.broadcast_to(x, (2, 3, 4))
array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]]])
```

The importance of broadcasting should be manifestly clear. We could now, say, compute the position of tens of thousands of particles relative to their center of mass, in a concise and highly-optimized way. That being said, there are rules that dictate how NumPy performs its broadcasting, and what conditions makes two arrays broadcast-compatible. We will spend the rest of this section discussing these rules and further developing our intuition for broadcasting.

***

**Reading Comprehension: Basic Broadcasting**

Generate a random array of 10,000 2D points using `np.random.rand`. Compute the "center of mass" of these points, which is simply the average x-coordinate and the average y-coordinate of these 10,000 points. Then, use broadcasting to compute the shape-(10000,2) array that stores the position of the points *relative* to the center of mass. For example, if the center of mass is $(0.5, 1)$, and the absolute position of a point is $(2, 3)$, then the position of that point *relative* to the center of mass is simply $(2, 3) - (0.5, 1) = (1.5, 2)$

***

## Rules of Broadcasting
Array broadcasting cannot accommodate arbitrary combinations of array shapes. For example, a (7,5)-shape array is incompatible with a shape-(11,3) array. Trying to add two such arrays would produce a `ValueError`. To determine if two arrays are broadcast-compatible, line up their shapes so that their trailing dimensions are aligned, and then check that each pair of aligned dimensions satisfy either of the following conditions:

- the aligned dimensions have the same size
- one of the dimensions has a size of 1

The two arrays are broadcast-compatible if either of these conditions are satisfied for each pair of aligned dimensions. It is okay to have one array with a higher-dimensionality and thus to have "dangling" leading dimensions. Any size-1 dimension or "missing" dimension will be filled-in by broadcasting the content of that array.

Taking our grade-book example from the preceding subsection, let's see that the shape-(6,3) and shape-(3,) arrays satisfy these rules for broadcast-compatibility:
```
     array-1: 6 x 3
     array-2:     3
result-shape: 6 x 3
```

Let's look an assortment of pairs of array-shapes and see whether or not they are broadcast-compatible:

```
     array-1:         8
     array-2: 5 x 2 x 8
result-shape: 5 x 2 x 8

     array-1:     5 x 2
     array-2: 5 x 4 x 2
result-shape: INCOMPATIBLE

     array-1:     4 x 2
     array-2: 5 x 4 x 2
result-shape: 5 x 4 x 2

     array-1: 8 x 1 x 3
     array-2: 8 x 5 x 3
result-shape: 8 x 5 x 3

     array-1: 5 x 1 x 3 x 2
     array-2:     9 x 1 x 2
result-shape: 5 x 9 x 3 x 2

     array-1: 1 x 3 x 2
     array-2:     8 x 2 
result-shape: INCOMPATIBLE

     array-1: 2 x 1
     array-2:     1  
result-shape: 2 x 1
```


***

**Reading Comprehension: Broadcast Compatibility**

Given the following pairs of array-shapes, determine what the resulting broadcasted shapes will be. Indicate if a pair is broadcast-incompatible. 

1\. `7 x 2` with `7`

2\. `4` with `3 x 4`

3\. `1 x 3 x 1` with `8 x 1 x 1`

4\. `9 x 2 x 5` with `2 x 5`

5\. `3` with `3 x 3 x 2`

***

## Size-1 Axes & The `newaxis` Object 
Here we will learn the utility of introducing size-1 dimensions into an array.

Dimensions of size-1 are special in that they can be broadcasted to any size necessary. Additionally, recalling the rules for reshaping an array, you can introduce size-1 dimensions to an array without changing the overall size (i.e. total number of entries in an array. Thus we are free to add size-1 dimensions to an array as-needed (we will see why this can be useful in a moment). Let's reshape a shape-(3,) array into a shape-(1, 3, 1, 1) array:
```python
>>> import numpy as np
>>> np.array([1, 2, 3]).reshape(1, 3, 1, 1)
array([[[[1]],

        [[2]],

        [[3]]]])
```

Thus the 1-D array with three entries has been reshaped to possess 4-dimensions: "one stack of three sheets, each containing a single row and column". There is another way to introduce size-1 dimensions. NumPy provides the `newaxis` object for this purpose. Let's immediately demonstrate how `np.newaxis` can be used:
```python
# demonstrating the usage of the `numpy.newaxis` object
>>> x= np.array([1, 2, 3])
>>> y= x[np.newaxis, :, np.newaxis, np.newaxis]
>>> y
array([[[[1]],

        [[2]],

        [[3]]]])
>>> y.shape
(1, 1, 3, 1)
```
See that including `np.newaxis` objects when indexing `x` returns a "view" of `x`, with size-1 dimensions inserted correspondingly. The resulting array is not a copy of `x`, it points to the exact same data as `x`, but merely with a different indexing layout. This is no different than what we achieved via reshaping.

Moving on to a more pressing matter: why would we ever want to introduce these spurious dimensions into an array? Let's take an example to demonstrate the utility of size-1 dimensions. Suppose that we want to multiply all possible pairs of entries between two arrays: `array([1, 2, 3])` with `array([[4, 5, 6, 7]])`. That is, we want to perform twelve multiplications, and have access to each result. At first glance, combining a shape-(3,) array with a shape-(4,) array seems inadmissible for broadcasting; we seem to be doomed to perform nested for-loops like a bunch of cavemen and cavewomen. Fortunately, we can make clever use of size-1 dimensions so that we can perform this computation in a vectorized way. 

Let's introduce size-1 dimensions into both arrays as so:

```python
# inserting size-1 dimensions into `x` and `y` in 
# preparation of broadcasting
>>> x_1d = np.array([1, 2, 3]) 
>>> x = x_1d.reshape(3, 1)
>>> x
array([[1],
       [2],
       [3]])

>>> y_1d = np.array([4, 5, 6, 7])
>>> y = y_1d.reshape(1, 4)
>>> y
array([[4, 5, 6, 7]])
```

`x` is now a shape-(1, 3) array and `y` is a shape-(4, 1) array. According to the broadcasting rules, these arrays are broadcast compatible and will multiply to produce a shape-(3, 4) array. Let's see that multiplying these two arrays will exactly produce the twelve numbers that we are after:

```python
# broadcast-multiplying `x` and `y`
>>> x * y
array([[ 4,  5,  6,  7],
       [ 8, 10, 12, 14],
       [12, 15, 18, 21]])
```

\begin{equation}
\left(
\begin{array}{*{1}{X}}
  \left(1\right) \\
  \left(2\right) \\
  \left(3\right)
\end{array} \right)
% 
\times \left(\left( \begin{array}{*{4}{X}}
  4 & 5 & 6 & 7
\end{array}\right)\right)
% 
\rightarrow \left( \begin{array}{*{4}{X}}
  1 & 1 & 1 & 1 \\
  2 & 2 & 2 & 2 \\
  3 & 3 & 3 & 3
\end{array}\right)
%
\times \left( \begin{array}{*{4}{X}}
  4 & 5 & 6 & 7 \\
  4 & 5 & 6 & 7 \\
  4 & 5 & 6 & 7
\end{array}\right)
%
= \left( \begin{array}{*{4}{X}}
  1\times4 & 1\times5 & 1\times6 & 1\times7 \\
  2\times4 & 2\times5 & 2\times6 & 2\times7 \\
  3\times4 & 3\times5 & 3\times6 & 3\times7
\end{array}\right)
\
\end{equation}

See that entry `(i, j)` of the resulting array corresponds to `x_1d[i] * y_1d[j]`. 

Through the use of simple reshaping, shrewdly inserting size-1 dimensions allowed us to coerce NumPy into performing exactly the combination-multiplication that we desired. Furthermore, a keen understanding of what broadcasting is provides us with a clear interpretation of the structure of the result of this calculation. That is, if I reshape `x` to be a shape-(M, 1) array, and I reshape `y` to be a shape-(1, N) array, then (according to broadcasting rules) `x * y` would produce a shape-(M, N) array storing the multiplication of each of `x`'s M numbers with each of `y`'s N numbers. 

***

**Reading Comprehension: Basic Broadcasting II**

Given the shape-(2,3,4) array:
```python
>>> x = np.array([[[ 0,  1,  2,  3],
                   [ 4,  5,  6,  7],
                   [ 8,  9, 10, 11]],

                  [[12, 13, 14, 15],
                   [16, 17, 18, 19],
                   [20, 21, 22, 23]]])
```

Normalize `x` such that *each of its rows, within each sheet, will sum to a value of 1*. Make use of the sequential function `np.sum`, which should be called only once, and broadcast-division.

***

***

**Reading Comprehension: Basic Broadcasting III**

A digital image is simply an array of numbers, which instructs a grid of pixels on a monitor to shine light of specific colors, according to the numerical values in that array. 

An RGB-image can thus be stored as a 3D NumPy array of shape-$(V, H, 3)$. $V$ is the number of pixels along the vertical direction, $H$ is the number of pixels along the horizontal, and the size-3 dimension stores the red, blue, and green color values for a given pixel. Thus a $(32, 32, 3)$ array would be a 32x32 RBG image.

You often work with a collection of images. Suppose we want to store N images in a single array; thus we now consider a 4D shape-(N, V, H, 3) array.  For the sake of convenience, let's simply generate a 4D-array of random numbers as a placeholder for real image data. We will generate 500, 48x48 RGB images:

```python
>>> images = np.random.rand(500, 48, 48, 3)

```

Using the sequential function `np.max` and broadcasting, normalize `images` such that the largest value within each color-channel of each image is 1.

***

## Reading Comprehension Solutions

**Basic Broadcasting: Solution**

Generating the random array of 10,000, 2D points, and their "center-of-mass".
```python
>>> pts = np.random.rand(10000, 2)
# find the mean x-coord and y-coord of the 10000 points
>>> center_of_mass = pts.mean(axis=0)  # -> array([mean_x, mean_y])
>>> center_of_mass.shape
(2,)

# Use broadcasting to compute the position of each point relative
# to the center of mass. The center of mass coordinates are subtracted
# from each of the 10000 points, via broadcast-subtraction
>>> relative_pos = pts - center_of_mass # shape-(10000,2) w/ shape-(2,)
>>> relative_pos.shape
(10000, 2)
```

**Broadcast Compatibility: Solution**

1\. Incompatible

2\. `3 x 4`

3\. `8 x 3 x 1`

4\. `9 x 2 x 5`

5\. Incompatible

**Basic Broadcasting II: Solution**

```python
# a shape-(2, 3, 4) array
>>> x = np.array([[[ 0,  1,  2,  3],
                   [ 4,  5,  6,  7],
                   [ 8,  9, 10, 11]],

                  [[12, 13, 14, 15],
                   [16, 17, 18, 19],
                   [20, 21, 22, 23]]])


# sum over each row, within a sheet
>>> summed_rows = x.sum(axis=1)
array([[12, 15, 18, 21],
       [48, 51, 54, 57]])

# this shape-(2,4) array can be broadcast-divided
# along the rows of `x`, if we insert a size-1 axis
>>> x_norm = x / summed_rows[:, np.newaxis, :]

# verifying the solution
>>> x_norm.sum(axis=1)
array([[ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.]])
```

**Basic Broadcasting III: Solution**

```python
# a collection of 500 48x48 RGB images
>>> images = np.random.rand(500, 48, 48, 3)

# finding the max-value within each color-channel of each image
>>> max_vals = images.max(axis=(1,2))
>>> max_vals.shape
(500, 3)

# we can insert size-1 dimensions so that we can
# broadcast-divide these max-values with
# the pixels of the images.
# broadcasting (500, 48, 48, 3) with (500, 1, 1, 3)
>>> normed_images = images / max_vals.reshape(500, 1, 1, 3)

# checking that all the max-values are 1
>>> normed_images.max(axis=(1,2))
array([[ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       ..., 
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.]])

# a rigorous check
>>> np.all(normed_images.max(axis=(1,2)) == 1)
True
```